In [3]:
import os, sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn as sklearn

In [4]:
patient_df = pd.read_csv('data/patient_df').drop(columns='Unnamed: 0')
mimic_patient_df = pd.read_csv('data/sepsis3-MIMICdata/sepsis3-df.csv')

In [6]:
patient_df.shape

(188453, 43)

## LR and SVM Baseline Models using different imputation approaches

In [1]:
from sklearn.impute import SimpleImputer

In [ ]:
imputer = SimpleImputer()